Los Angeles Crime 2020 - Present
===================
This dataset reflects incidents of crime in the City of Los Angeles dating back to 2020. This data is transcribed from original crime reports that are typed on paper and therefore there may be some inaccuracies within the data. 

**`Acknowledgements:`**
CSV downloaded from https://www.kaggle.com/datasets/chaitanyakck/crime-data-from-2020-to-present?select=Crime_Data_from_2020_to_Present.csv

Data Collection Source: https://data.lacity.org/

**`Project Goal`**
------------
Categorize the neighborhoods of Los Angeles by crime severity to know and predict which areas/municipalities are categorized as dangerous according to the 

`Client Story:` A Los Angeles-based realtor wants to know which areas of Los Anegeles to invest in (buy real estate). Since safety is a major concern for potential customers, the real estate agency wants to know the safest municipalities in the city to do business in. Thus, the real estate agency prefers to avoid areas of Los Angeles that may be risky to do business in. Areas with higher crime may scare away customers which could lead to a net loss of the investment. Market real estate price is not an issue to be wary of, the real estate agency solely wants to focus on the crime levels of the municipalities within the city. Important to note that the project goal is not solely to define the municipalities in the present but predict the potential crime levels for the future. This ensures that the real estate investment is worthwhile in the long run and has net gains over an extended period of time. 


Frame the Problem and Look at the Big Picture
=====================================

1. **Define the objective in business terms:** 

2. **How will your solution be used?**

3. **What are current solutions (if any)?** 

4. **How should you frame this problem?**

5. **How should performance be measured? Is the performance measure aligned with the business objective?** 

6. **What would be the minimum performance needed to reach the business objective?**

7. **What are comparable problems? Can you reuse experience or tools?** 

8. **Is human expertise available?**

9. **How would you solve the problem manually?** 

10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 

Get The Data
==================